# Setting

### Note : the file is very large. So i will manage one by one.

In [1]:
#importing some lib and set display table.
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [2]:
# the name of file meaning as below. 
# ac = account ,b = base, st = statement
# tr = Transaction ,ct = Contacting
# b_ac = b join with ac by using AC and B analysis file.
# b_ac _st = b_ac join with st by using  ST analysis file.
ac_1218 = 'Customer_RL_Account_1218.txt'
ac_0119 = 'Customer_RL_Account_0119.txt'
ac_0219 = 'Customer_RL_Account_0219.txt'
b_1218 = 'Customer_RL_Base_1218.txt'
b_0119 = 'Customer_RL_Base_0119.txt'
b_0219 = 'Customer_RL_Base_0219.txt'
st_1218 = 'Customer_RL_Statement_1218.txt'
st_0119 = 'Customer_RL_Statement_0119.txt'
st_0219 = 'Customer_RL_Statement_0219.txt'
tr_1218 = 'Customer_RL_Transaction_1218.txt'
tr_0119 = 'Customer_RL_Transaction_0119.txt'
tr_0219 = 'Customer_RL_Transaction_0219.txt'
ct_1218 = 'Customer_RL_Contacting_1218.txt'
ct_0119 = 'Customer_RL_Contacting_0119.txt'
ct_0219 = 'Customer_RL_Contacting_0219.txt'
b_ac_1218 = 'b_ac_1218_data.csv'
b_ac_0119 = 'b_ac_0119_data.csv'
b_ac_0219 = 'b_ac_0119_data.csv'
b_ac_st_1218 = 'b_ac_st_1218_data.csv'
b_ac_st_0119 = 'b_ac_st_0119_data.csv'
b_ac_st_0219 = 'b_ac_st_0219_data.csv'


# Transaction Analysis

In [3]:
# Statement 
tr_data = pd.read_csv(tr_0219,header = 0,sep = '|',encoding="ISO-8859-1")

In [4]:
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19864457 entries, 0 to 19864456
Data columns (total 11 columns):
CUSTOMER_ID         int64
Card_no             object
Txn_Date            int64
TXN_Amount          float64
transaction_type    object
Debit_Credit        object
cyc_period          float64
MERCHANT_A_NAME     object
MCC                 float64
COUNTRY_CODE        object
MIS_MTH             int64
dtypes: float64(3), int64(3), object(5)
memory usage: 1.6+ GB


In [5]:
tr_data.transaction_type.unique()

array(['Payment', 'Cash Adv', 'Undefine', 'First Draw'], dtype=object)

In [6]:
# Casting type and fill&drop NAN data.

tr_data['CUSTOMER_ID'] = tr_data['CUSTOMER_ID'].astype('str')
tr_data['Card_no'] = tr_data['Card_no'].astype('str')
tr_data['Txn_Date'] = tr_data['Txn_Date'].astype('str')
tr_data['transaction_type'] = tr_data['transaction_type'].astype('str')
tr_data['Debit_Credit'] = tr_data['Debit_Credit'].astype('str')
tr_data['TXN_Amount'] = tr_data['TXN_Amount'].astype('float')
tr_data['cyc_period'] = tr_data['cyc_period'].astype('str')
tr_data = tr_data.drop(columns = ['MERCHANT_A_NAME','MIS_MTH','COUNTRY_CODE','MCC'])

## Payment

In [7]:
payment = tr_data[tr_data['transaction_type']=='Payment']

In [8]:
# percentile function
def Payment_percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'Payment_PT_%s' % n
    return percentile_
payment=payment.groupby(['CUSTOMER_ID','Card_no']).TXN_Amount.agg([np.size,np.mean,np.std,np.min,Payment_percentile(5),Payment_percentile(25),Payment_percentile(50),Payment_percentile(75),Payment_percentile(95),np.max,np.sum]).rename(columns={'mean':'Payment_Mean','std':'Payment_Std','size':'Payment_Size','amin':'Payment_Min','amax':'Payment_Max','sum':'Payment_sum'})
payment = payment.reset_index()
payment.head()

,CUSTOMER_ID,Card_no,Payment_Size,Payment_Mean,Payment_Std,Payment_Min,Payment_PT_5,Payment_PT_25,Payment_PT_50,Payment_PT_75,Payment_PT_95,Payment_Max,Payment_sum
0,1000000000000019,421668XXXXXX1467,28.0,-3607.142857,1091.695444,-5500.0,-5000.0,-4250.0,-3500.0,-3000.0,-2000.0,-1500.0,-101000.0
1,1000000000000059,421668XXXXXX6559,24.0,-12362.500000,3576.075393,-17550.0,-17550.0,-14137.5,-13000.0,-10000.0,-7550.0,-7550.0,-296700.0
2,1000000000000137,421668XXXXXX8139,23.0,-1786.565217,74.645725,-2000.0,-1892.4,-1800.0,-1800.0,-1743.5,-1701.5,-1600.0,-41091.0
3,1000000000000190,421668XXXXXX8672,24.0,-5562.500000,5294.198014,-30000.0,-5000.0,-5000.0,-5000.0,-4375.0,-3000.0,-1000.0,-133500.0
4,1000000000000324,421668XXXXXX5069,11.0,-55142.909091,93124.790762,-319068.0,-212380.5,-51708.5,-22000.0,-6050.0,-2147.0,-294.0,-606572.0


In [9]:
from sklearn.preprocessing import StandardScaler
fit_data = payment[['Payment_Min','Payment_PT_5','Payment_PT_25','Payment_PT_50','Payment_PT_75','Payment_PT_95','Payment_Max']]
scaler = StandardScaler()
scaler = scaler.fit(fit_data)
print("The mean of standardize::", scaler.mean_)
print("The variance of standardize::", scaler.var_ )
fit_data = pd.DataFrame(scaler.transform(fit_data))
payment['Payment_Min_Score'] = fit_data[0]
payment['Payment_PT_5_Score'] = fit_data[1]
payment['Payment_PT_25_Score'] = fit_data[2]
payment['Payment_PT_50_Score'] = fit_data[3]
payment['Payment_PT_75_Score'] = fit_data[4]
payment['Payment_PT_95_Score'] = fit_data[5]
payment['Payment_Max_Score'] = fit_data[6]

The mean of standardize:: [-21112.13727509 -13765.70043801  -6707.12646377  -4581.25400571
  -3516.83769733  -2667.40386171  -2278.025108  ]
The variance of standardize:: [8.97236202e+08 4.20005499e+08 1.34664300e+08 7.28755835e+07
 5.11554759e+07 4.51101228e+07 4.95350637e+07]


In [10]:
payment['Payment_Mean_Score'] =  payment[['Payment_Min_Score','Payment_PT_5_Score','Payment_PT_25_Score','Payment_PT_50_Score','Payment_PT_75_Score','Payment_PT_95_Score','Payment_Max_Score']].mean(axis=1)
payment['Payment_Grade'] = 100 - (((payment['Payment_Mean_Score'] - payment['Payment_Mean_Score'].min())/(payment['Payment_Mean_Score'].max() - payment['Payment_Mean_Score'].min()))*100)

In [11]:
payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696570 entries, 0 to 696569
Data columns (total 22 columns):
CUSTOMER_ID            696570 non-null object
Card_no                696570 non-null object
Payment_Size           696570 non-null float64
Payment_Mean           696570 non-null float64
Payment_Std            634565 non-null float64
Payment_Min            696570 non-null float64
Payment_PT_5           696570 non-null float64
Payment_PT_25          696570 non-null float64
Payment_PT_50          696570 non-null float64
Payment_PT_75          696570 non-null float64
Payment_PT_95          696570 non-null float64
Payment_Max            696570 non-null float64
Payment_sum            696570 non-null float64
Payment_Min_Score      696570 non-null float64
Payment_PT_5_Score     696570 non-null float64
Payment_PT_25_Score    696570 non-null float64
Payment_PT_50_Score    696570 non-null float64
Payment_PT_75_Score    696570 non-null float64
Payment_PT_95_Score    696570 non-null float6

## Cash Adv

In [12]:
cash = tr_data[tr_data['transaction_type']=='Cash Adv']

In [13]:
# percentile function
def Cash_percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'Cash_PT_%s' % n
    return percentile_
cash=cash.groupby(['CUSTOMER_ID','Card_no']).TXN_Amount.agg([np.size,np.mean,np.std,np.min,Cash_percentile(5),Cash_percentile(25),Cash_percentile(50),Cash_percentile(75),Cash_percentile(95),np.max,np.sum]).rename(columns={'mean':'Cash_Mean','std':'Cash_Std','size':'Cash_Size','amin':'Cash_Min','amax':'Cash_Max','sum':'Cash_sum'})
cash = cash.reset_index()
cash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517716 entries, 0 to 517715
Data columns (total 13 columns):
CUSTOMER_ID    517716 non-null object
Card_no        517716 non-null object
Cash_Size      517716 non-null float64
Cash_Mean      517716 non-null float64
Cash_Std       472181 non-null float64
Cash_Min       517716 non-null float64
Cash_PT_5      517716 non-null float64
Cash_PT_25     517716 non-null float64
Cash_PT_50     517716 non-null float64
Cash_PT_75     517716 non-null float64
Cash_PT_95     517716 non-null float64
Cash_Max       517716 non-null float64
Cash_sum       517716 non-null float64
dtypes: float64(11), object(2)
memory usage: 51.3+ MB


In [14]:
from sklearn.preprocessing import StandardScaler
fit_data = cash[['Cash_Min','Cash_PT_5','Cash_PT_25','Cash_PT_50','Cash_PT_75','Cash_PT_95','Cash_Max']]
scaler = StandardScaler()
scaler = scaler.fit(fit_data)
print("The mean of standardize::", scaler.mean_)
print("The variance of standardize::", scaler.var_ )
fit_data = pd.DataFrame(scaler.transform(fit_data))
cash['Cash_Min_Score'] = fit_data[0]
cash['Cash_PT_5_Score'] = fit_data[1]
cash['Cash_PT_25_Score'] = fit_data[2]
cash['Cash_PT_50_Score'] = fit_data[3]
cash['Cash_PT_75_Score'] = fit_data[4]
cash['Cash_PT_95_Score'] = fit_data[5]
cash['Cash_Max_Score'] = fit_data[6]

The mean of standardize:: [ 3278.14669838  3602.30629209  4737.74874528  6336.08878913
  8628.52027    11998.17196405 13516.15414453]
The variance of standardize:: [3.73635918e+07 3.78029568e+07 4.56233614e+07 5.85918229e+07
 7.44220010e+07 1.00323205e+08 1.31868889e+08]


In [15]:
cash['Cash_Mean_Score'] =  cash[['Cash_Min_Score','Cash_PT_5_Score','Cash_PT_25_Score','Cash_PT_50_Score','Cash_PT_75_Score','Cash_PT_95_Score','Cash_Max_Score']].mean(axis=1)
cash['Cash_Grade'] = ((cash['Cash_Mean_Score'] - cash['Cash_Mean_Score'].min())/(cash['Cash_Mean_Score'].max() - cash['Cash_Mean_Score'].min()))*100

In [16]:
cash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517716 entries, 0 to 517715
Data columns (total 22 columns):
CUSTOMER_ID         517716 non-null object
Card_no             517716 non-null object
Cash_Size           517716 non-null float64
Cash_Mean           517716 non-null float64
Cash_Std            472181 non-null float64
Cash_Min            517716 non-null float64
Cash_PT_5           517716 non-null float64
Cash_PT_25          517716 non-null float64
Cash_PT_50          517716 non-null float64
Cash_PT_75          517716 non-null float64
Cash_PT_95          517716 non-null float64
Cash_Max            517716 non-null float64
Cash_sum            517716 non-null float64
Cash_Min_Score      517716 non-null float64
Cash_PT_5_Score     517716 non-null float64
Cash_PT_25_Score    517716 non-null float64
Cash_PT_50_Score    517716 non-null float64
Cash_PT_75_Score    517716 non-null float64
Cash_PT_95_Score    517716 non-null float64
Cash_Max_Score      517716 non-null float64
Cash_Mean_S

# Merge Data

In [17]:
b_ac_st_data = pd.read_csv(b_ac_st_0219,index_col=0,encoding="utf-8-sig") 
b_ac_st_data['CUSTOMER_ID'] = b_ac_st_data['CUSTOMER_ID'].astype('str')
b_ac_st_data['CARD_NO'] = b_ac_st_data['CARD_NO'].astype('str')
b_ac_st_data['CARD_NO_Last_4_digit'] = b_ac_st_data['CARD_NO'].str.slice(-5,-1)
payment['CARD_NO_Last_4_digit'] = payment['Card_no'].str.slice(-4)
cash['CARD_NO_Last_4_digit'] = cash['Card_no'].str.slice(-4)
b_ac_st_data['Account_Number'] = b_ac_st_data['Account_Number'].astype('str')
b_ac_st_data = b_ac_st_data.drop(b_ac_st_data[b_ac_st_data['Customer_Status'] =='Inactive'].index, axis=0)
b_ac_st_data = b_ac_st_data.drop(b_ac_st_data[b_ac_st_data['Account_Status'] =='Deactivate Card'].index, axis=0)
b_ac_st_data.info()

G:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 526709 entries, 0 to 931413
Data columns (total 46 columns):
CUSTOMER_ID                   526709 non-null object
DOB                           521145 non-null float64
Gender                        521126 non-null object
MONTHLY_INCOME_RL             521145 non-null float64
Marital_status                521145 non-null object
Mobilephone_fg                521145 non-null object
Officephone_fg                521145 non-null object
Homephone_fg                  521145 non-null object
NO_OF_CHILD                   521145 non-null float64
Occupation                    521145 non-null object
Province                      521095 non-null object
Zipcode                       521145 non-null object
Total_Credit_Limit            521145 non-null float64
Customer_Status               521145 non-null object
Is_Staff                      521145 non-null object
Is_KTC_Credit_Card_Holder     521145 non-null object
Age                           521145 

In [18]:
total_PC = payment.merge(cash, left_on=['CUSTOMER_ID','Card_no','CARD_NO_Last_4_digit'], right_on=['CUSTOMER_ID','Card_no','CARD_NO_Last_4_digit'], how ='inner').sort_values(['CUSTOMER_ID'])
total_PC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513002 entries, 0 to 513001
Data columns (total 43 columns):
CUSTOMER_ID             513002 non-null object
Card_no                 513002 non-null object
Payment_Size            513002 non-null float64
Payment_Mean            513002 non-null float64
Payment_Std             503699 non-null float64
Payment_Min             513002 non-null float64
Payment_PT_5            513002 non-null float64
Payment_PT_25           513002 non-null float64
Payment_PT_50           513002 non-null float64
Payment_PT_75           513002 non-null float64
Payment_PT_95           513002 non-null float64
Payment_Max             513002 non-null float64
Payment_sum             513002 non-null float64
Payment_Min_Score       513002 non-null float64
Payment_PT_5_Score      513002 non-null float64
Payment_PT_25_Score     513002 non-null float64
Payment_PT_50_Score     513002 non-null float64
Payment_PT_75_Score     513002 non-null float64
Payment_PT_95_Score     513

In [19]:
Total_data = b_ac_st_data.merge(total_PC, left_on=['CUSTOMER_ID','CARD_NO_Last_4_digit'], right_on=['CUSTOMER_ID','CARD_NO_Last_4_digit'], how ='inner').sort_values(['CUSTOMER_ID'])

In [20]:
Total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332888 entries, 0 to 332887
Data columns (total 87 columns):
CUSTOMER_ID                   332888 non-null object
DOB                           332455 non-null float64
Gender                        332444 non-null object
MONTHLY_INCOME_RL             332455 non-null float64
Marital_status                332455 non-null object
Mobilephone_fg                332455 non-null object
Officephone_fg                332455 non-null object
Homephone_fg                  332455 non-null object
NO_OF_CHILD                   332455 non-null float64
Occupation                    332455 non-null object
Province                      332427 non-null object
Zipcode                       332455 non-null object
Total_Credit_Limit            332455 non-null float64
Customer_Status               332455 non-null object
Is_Staff                      332455 non-null object
Is_KTC_Credit_Card_Holder     332455 non-null object
Age                           332455 

In [21]:
Total_data

,CUSTOMER_ID,DOB,Gender,MONTHLY_INCOME_RL,Marital_status,Mobilephone_fg,Officephone_fg,Homephone_fg,NO_OF_CHILD,Occupation,Province,Zipcode,Total_Credit_Limit,Customer_Status,Is_Staff,Is_KTC_Credit_Card_Holder,Age,Account_Number,Account_Status,Account_Type,Open_Date,Credit_Limit,Current_Balance,Card_Age,CARD_NO,CyC_Balance_Size,CyC_Balance_Min,CyC_Balance_Max,CyC_Balance_Mean,Cyc_Balance_Std,Cyc_Balance_PT_5,Cyc_Balance_PT_25,Cyc_Balance_PT_50,Cyc_Balance_PT_75,Cyc_Balance_PT_95,CyC_Balance_Min_Score,Cyc_Balance_PT_5_Score,Cyc_Balance_PT_25_Score,Cyc_Balance_PT_50_Score,Cyc_Balance_PT_75_Score,Cyc_Balance_PT_95_Score,Cyc_Balance_PT_Max_Score,Cyc_Balance_Mean_Score,Cyc_Balance_Grade,Cyc_Balance_Behavier_Group,CARD_NO_Last_4_digit,Card_no,Payment_Size,Payment_Mean,Payment_Std,Payment_Min,Payment_PT_5,Payment_PT_25,Payment_PT_50,Payment_PT_75,Payment_PT_95,Payment_Max,Payment_sum,Payment_Min_Score,Payment_PT_5_Score,Payment_PT_25_Score,Payment_PT_50_Score,Payment_PT_75_Score,Payment_PT_95_Score,Payment_Max_Score,Payment_Mean_Score,Payment_Grade,Cash_Size,Cash_Mean,Cash_Std,Cash_Min,Cash_PT_5,Cash_PT_25,Cash_PT_50,Cash_PT_75,Cash_PT_95,Cash_Max,Cash_sum,Cash_Min_Score,Cash_PT_5_Score,Cash_PT_25_Score,Cash_PT_50_Score,Cash_PT_75_Score,Cash_PT_95_Score,Cash_Max_Score,Cash_Mean_Score,Cash_Grade
0,1000000000000019,19611204.0,Male,80040.0,Married,Y,Y,N,0.0,55,Roi Et,45000,0.0,Active,N,Y,58.0,4216680001222036.0,Normal,Revolving Loan Card,20110503.0,50000.0,40622.85,8.0,4216680008814676,24.0,41465.00,50975.69,47525.812917,3328.910511,42281.1835,44175.0950,49498.505,50048.3875,50958.7945,1.058805,0.982971,0.738166,0.653178,0.467876,0.342465,0.299074,0.648933,78.795585,9,1467,421668XXXXXX1467,28.0,-3607.142857,1091.695444,-5500.00,-5000.0000,-4250.000,-3500.000,-3000.0000,-2000.0000,-1500.00,-101000.00,0.521205,0.427720,0.211739,0.126659,0.072262,0.099369,0.110545,0.224214,10.544513,28.0,2464.285714,1035.737092,1000.0,1000.0,2000.0,3000.0,3000.0,4000.0,5000.0,69000.00,-0.372698,-0.423249,-0.405322,-0.435832,-0.652444,-0.798528,-0.741604,-0.547097,28.917140
1,1000000000000059,19640211.0,Male,123211.0,Married,Y,Y,Y,0.0,Not Answer,Surat Thani,84290,0.0,Active,Y,Y,55.0,4216680001176531.0,Normal,Revolving Loan Card,20101201.0,450000.0,174559.55,9.0,4216680008165590,24.0,85003.14,227363.84,153715.259167,45654.761291,89999.1420,112922.0975,161871.050,189929.0575,217437.8355,2.739177,2.749198,2.926535,3.795063,3.953585,4.148198,4.232992,3.506392,73.985582,2,6559,421668XXXXXX6559,24.0,-12362.500000,3576.075393,-17550.00,-17550.0000,-14137.500,-13000.000,-10000.0000,-7550.0000,-7550.00,-296700.00,0.118921,-0.184654,-0.640301,-0.986180,-0.906444,-0.726965,-0.749061,-0.582098,11.942594,13.0,17769.230769,5746.793978,1000.0,6400.0,20000.0,20000.0,20000.0,20000.0,20000.0,231000.00,-0.372698,0.455027,2.259566,1.785075,1.318154,0.798893,0.564627,0.972663,30.591308
2,1000000000000190,19690304.0,Male,30190.0,Married,N,Y,N,1.0,Permanent Employee,Chiang Mai,50100,0.0,Active,N,Y,50.0,4216680001107009.0,Normal,Revolving Loan Card,20100419.0,150900.0,130830.46,9.0,4216680007186720,24.0,-37791.30,145961.49,116020.454583,45332.112392,12063.9485,108415.2800,134404.635,138105.8375,144146.0300,-2.000124,-0.135486,2.783073,3.027114,2.662194,2.472737,2.417506,1.603859,77.188145,4,8672,421668XXXXXX8672,24.0,-5562.500000,5294.198014,-30000.00,-5000.0000,-5000.000,-5000.000,-4375.0000,-3000.0000,-1000.00,-133500.00,-0.296718,0.427720,0.147109,-0.049052,-0.119984,-0.049520,0.181586,0.034449,10.873552,22.0,11077.272727,5903.020358,3000.0,3100.0,6250.0,10000.0,15775.0,20000.0,20000.0,243700.00,-0.045504,-0.081697,0.223888,0.478659,0.828403,0.798893,0.564627,0.395324,29.955311
3,1000000000000324,19640904.0,Male,550806.0,Single,Y,N,Y,0.0,Doctor / Dentist,Chonburi,20150,0.0,Active,N,Y,55.0,4216680000949436.0,Normal,Revolving Loan Card,20071228.0,500000.0,346611.89,12.0,4216680009350696,24.0,-0.01,368611.89,93079.153333,127489.758190,-0.0100,0.0000,293.990,198082.1650,355726.7440

In [22]:
Total_data.sort_values(by=['Cyc_Balance_Grade','Payment_Grade','Cash_Grade'],ascending=False)

,CUSTOMER_ID,DOB,Gender,MONTHLY_INCOME_RL,Marital_status,Mobilephone_fg,Officephone_fg,Homephone_fg,NO_OF_CHILD,Occupation,Province,Zipcode,Total_Credit_Limit,Customer_Status,Is_Staff,Is_KTC_Credit_Card_Holder,Age,Account_Number,Account_Status,Account_Type,Open_Date,Credit_Limit,Current_Balance,Card_Age,CARD_NO,CyC_Balance_Size,CyC_Balance_Min,CyC_Balance_Max,CyC_Balance_Mean,Cyc_Balance_Std,Cyc_Balance_PT_5,Cyc_Balance_PT_25,Cyc_Balance_PT_50,Cyc_Balance_PT_75,Cyc_Balance_PT_95,CyC_Balance_Min_Score,Cyc_Balance_PT_5_Score,Cyc_Balance_PT_25_Score,Cyc_Balance_PT_50_Score,Cyc_Balance_PT_75_Score,Cyc_Balance_PT_95_Score,Cyc_Balance_PT_Max_Score,Cyc_Balance_Mean_Score,Cyc_Balance_Grade,Cyc_Balance_Behavier_Group,CARD_NO_Last_4_digit,Card_no,Payment_Size,Payment_Mean,Payment_Std,Payment_Min,Payment_PT_5,Payment_PT_25,Payment_PT_50,Payment_PT_75,Payment_PT_95,Payment_Max,Payment_sum,Payment_Min_Score,Payment_PT_5_Score,Payment_PT_25_Score,Payment_PT_50_Score,Payment_PT_75_Score,Payment_PT_95_Score,Payment_Max_Score,Payment_Mean_Score,Payment_Grade,Cash_Size,Cash_Mean,Cash_Std,Cash_Min,Cash_PT_5,Cash_PT_25,Cash_PT_50,Cash_PT_75,Cash_PT_95,Cash_Max,Cash_sum,Cash_Min_Score,Cash_PT_5_Score,Cash_PT_25_Score,Cash_PT_50_Score,Cash_PT_75_Score,Cash_PT_95_Score,Cash_Max_Score,Cash_Mean_Score,Cash_Grade
29025,1000000000823425,19750826.0,Male,32000.00000,Single,Y,Y,Y,0.0,Other,Bangkok,10310,0.0,Active,N,Y,44.0,4216680000707132.0,Normal,Revolving Loan Card,20060904.0,160000.0,149559.62,13.0,4216680002739549,3.0,-319993.24,-12438.31,-1.717096e+05,154071.592906,-3.062636e+05,-2.513453e+05,-182697.310,-9.756781e+04,-2.946421e+04,-12.891821,-11.918027,-8.668899,-5.838912,-3.210597,-1.496016,-1.115225,-6.448500,90.742801,0,3954,421668XXXXXX3954,5.0,-116076.216000,123070.626489,-250000.00,-250000.0000,-250000.000,-45000.00,-30000.000,-10304.8640,-5381.08,-580381.08,-7.641337,-11.526978,-20.965400,-4.734693,-3.702745,-1.137135,-0.440893,-7.164169,23.355383,7.0,20000.000000,0.000000,20000.00,20000.000,20000.0000,20000.00,20000.0000,20000.0000,20000.00,140000.00,2.735647,2.666981,2.259566,1.785075,1.318154,0.798893,0.564627,1.732706,31.428571
23267,1000000000672103,19740107.0,Male,23333.00000,Single,Y,N,N,0.0,Permanent Employee,Pathum Thani,12110,0.0,Active,N,Y,45.0,4216680001269517.0,Normal,Revolving Loan Card,20110905.0,116600.0,-3.66,8.0,4216680009474215,24.0,-430049.21,28350.87,-1.128991e+04,89702.376372,-5.529227e+03,-1.504750e+01,1699.810,1.174476e+04,2.714271e+04,-17.139476,-0.786678,-0.668504,-0.683252,-0.486619,-0.201974,-0.205519,-2.881717,84.738783,0,7421,421668XXXXXX7421,71.0,-17090.989577,47216.335014,-400000.00,-32750.0000,-19000.000,-9000.00,-3329.360,-730.0000,-13.80,-1213460.26,-12.649032,-0.926333,-1.059320,-0.517616,0.026212,0.288458,0.321709,-2.073703,14.528920,41.0,15456.097561,6323.134064,1000.00,5000.000,10000.0000,20000.00,20000.0000,20000.0000,20000.00,633700.00,-0.372698,0.227326,0.779073,1.785075,1.318154,0.798893,0.564627,0.728636,30.322487
86140,1000000003040713,19861015.0,Female,19888.00000,Single,Y,Y,N,0.0,Nurse,Phitsanulok,65170,0.0,Active,N,Y,33.0,4216680001335257.0,Normal,Revolving Loan Card,20120611.0,59900.0,57988.22,7.0,4216680010536192,24.0,-354630.32,60832.99,3.922473e+03,81019.644778,-1.010995e+02,-7.707500e+00,-0.070,5.148643e+04,6.039071e+04,-14.228653,-0.585762,-0.668270,-0.730780,0.503710,0.558079,0.518917,-2.090394,83.406736,8,3619,421668XXXXXX3619,12.0,-36405.916667,112723.316004,-394001.00,-187953.9500,-3525.000,-2000.00,-1900.000,-1800.0000,-1800.00,-436871.00,-12.448758,-8.499461,0.274215,0.302371,0.226058,0.129147,0.067919,-2.849787,15.874587,17.0,28417.647059,84312.975242,2000.00,2000.000,3000.0000,5000.00,15000.0000,86920.0000,354600.00,483100.00,-0.209101,-0.260605,-0.257272,-0.174549,0.738567,7.480105,29.702293,5.288491,35.345626
18230,1000000000547856,19700709.0,Male,29167.00000,Married,Y,N,N,0.0,Other,Pathum Thani,12130,0.0,Active,N,Y,49.0,4216680001203018.0,Normal,Revolving Loan Card,20110228.0,11

In [23]:
Total_data = Total_data.sort_values(['CUSTOMER_ID'])
Total_data.to_csv("b_ac_st_t_0219_data.csv", encoding="utf-8-sig")

In [24]:
Total_data

,CUSTOMER_ID,DOB,Gender,MONTHLY_INCOME_RL,Marital_status,Mobilephone_fg,Officephone_fg,Homephone_fg,NO_OF_CHILD,Occupation,Province,Zipcode,Total_Credit_Limit,Customer_Status,Is_Staff,Is_KTC_Credit_Card_Holder,Age,Account_Number,Account_Status,Account_Type,Open_Date,Credit_Limit,Current_Balance,Card_Age,CARD_NO,CyC_Balance_Size,CyC_Balance_Min,CyC_Balance_Max,CyC_Balance_Mean,Cyc_Balance_Std,Cyc_Balance_PT_5,Cyc_Balance_PT_25,Cyc_Balance_PT_50,Cyc_Balance_PT_75,Cyc_Balance_PT_95,CyC_Balance_Min_Score,Cyc_Balance_PT_5_Score,Cyc_Balance_PT_25_Score,Cyc_Balance_PT_50_Score,Cyc_Balance_PT_75_Score,Cyc_Balance_PT_95_Score,Cyc_Balance_PT_Max_Score,Cyc_Balance_Mean_Score,Cyc_Balance_Grade,Cyc_Balance_Behavier_Group,CARD_NO_Last_4_digit,Card_no,Payment_Size,Payment_Mean,Payment_Std,Payment_Min,Payment_PT_5,Payment_PT_25,Payment_PT_50,Payment_PT_75,Payment_PT_95,Payment_Max,Payment_sum,Payment_Min_Score,Payment_PT_5_Score,Payment_PT_25_Score,Payment_PT_50_Score,Payment_PT_75_Score,Payment_PT_95_Score,Payment_Max_Score,Payment_Mean_Score,Payment_Grade,Cash_Size,Cash_Mean,Cash_Std,Cash_Min,Cash_PT_5,Cash_PT_25,Cash_PT_50,Cash_PT_75,Cash_PT_95,Cash_Max,Cash_sum,Cash_Min_Score,Cash_PT_5_Score,Cash_PT_25_Score,Cash_PT_50_Score,Cash_PT_75_Score,Cash_PT_95_Score,Cash_Max_Score,Cash_Mean_Score,Cash_Grade
0,1000000000000019,19611204.0,Male,80040.0,Married,Y,Y,N,0.0,55,Roi Et,45000,0.0,Active,N,Y,58.0,4216680001222036.0,Normal,Revolving Loan Card,20110503.0,50000.0,40622.85,8.0,4216680008814676,24.0,41465.00,50975.69,47525.812917,3328.910511,42281.1835,44175.0950,49498.505,50048.3875,50958.7945,1.058805,0.982971,0.738166,0.653178,0.467876,0.342465,0.299074,0.648933,78.795585,9,1467,421668XXXXXX1467,28.0,-3607.142857,1091.695444,-5500.00,-5000.0000,-4250.000,-3500.000,-3000.0000,-2000.0000,-1500.00,-101000.00,0.521205,0.427720,0.211739,0.126659,0.072262,0.099369,0.110545,0.224214,10.544513,28.0,2464.285714,1035.737092,1000.0,1000.0,2000.0,3000.0,3000.0,4000.0,5000.0,69000.00,-0.372698,-0.423249,-0.405322,-0.435832,-0.652444,-0.798528,-0.741604,-0.547097,28.917140
1,1000000000000059,19640211.0,Male,123211.0,Married,Y,Y,Y,0.0,Not Answer,Surat Thani,84290,0.0,Active,Y,Y,55.0,4216680001176531.0,Normal,Revolving Loan Card,20101201.0,450000.0,174559.55,9.0,4216680008165590,24.0,85003.14,227363.84,153715.259167,45654.761291,89999.1420,112922.0975,161871.050,189929.0575,217437.8355,2.739177,2.749198,2.926535,3.795063,3.953585,4.148198,4.232992,3.506392,73.985582,2,6559,421668XXXXXX6559,24.0,-12362.500000,3576.075393,-17550.00,-17550.0000,-14137.500,-13000.000,-10000.0000,-7550.0000,-7550.00,-296700.00,0.118921,-0.184654,-0.640301,-0.986180,-0.906444,-0.726965,-0.749061,-0.582098,11.942594,13.0,17769.230769,5746.793978,1000.0,6400.0,20000.0,20000.0,20000.0,20000.0,20000.0,231000.00,-0.372698,0.455027,2.259566,1.785075,1.318154,0.798893,0.564627,0.972663,30.591308
2,1000000000000190,19690304.0,Male,30190.0,Married,N,Y,N,1.0,Permanent Employee,Chiang Mai,50100,0.0,Active,N,Y,50.0,4216680001107009.0,Normal,Revolving Loan Card,20100419.0,150900.0,130830.46,9.0,4216680007186720,24.0,-37791.30,145961.49,116020.454583,45332.112392,12063.9485,108415.2800,134404.635,138105.8375,144146.0300,-2.000124,-0.135486,2.783073,3.027114,2.662194,2.472737,2.417506,1.603859,77.188145,4,8672,421668XXXXXX8672,24.0,-5562.500000,5294.198014,-30000.00,-5000.0000,-5000.000,-5000.000,-4375.0000,-3000.0000,-1000.00,-133500.00,-0.296718,0.427720,0.147109,-0.049052,-0.119984,-0.049520,0.181586,0.034449,10.873552,22.0,11077.272727,5903.020358,3000.0,3100.0,6250.0,10000.0,15775.0,20000.0,20000.0,243700.00,-0.045504,-0.081697,0.223888,0.478659,0.828403,0.798893,0.564627,0.395324,29.955311
3,1000000000000324,19640904.0,Male,550806.0,Single,Y,N,Y,0.0,Doctor / Dentist,Chonburi,20150,0.0,Active,N,Y,55.0,4216680000949436.0,Normal,Revolving Loan Card,20071228.0,500000.0,346611.89,12.0,4216680009350696,24.0,-0.01,368611.89,93079.153333,127489.758190,-0.0100,0.0000,293.990,198082.1650,355726.7440